In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
datapath = '/content/gdrive/My Drive/NLP/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint
from keras.layers import Concatenate, Input, Reshape, Conv2D, Dropout, Conv1D, CuDNNLSTM, CuDNNGRU, Bidirectional
from keras.models import Model, load_model

from keras.layers.core import *
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
import keras.backend as K
from keras import optimizers
import keras
from sklearn.feature_extraction.text import TfidfVectorizer

Using TensorFlow backend.


In [0]:
def create_data(file):
  text = file.read()
  fields = ['#','$','%', '^','&','*','@','/','//','\\','[',']','(',')', 'https','http','.','co','....','..']
  entries = [j.split('\n') for j in text.split('\n\n')]
  data = {'text':[], 'lang':[], 'tag':[]}
  for entry in entries:
    sent = entry[0].split('\t')[-1]
    if sent == 'positive':
      data['tag'].append(0)
    elif sent=='negative':
      data['tag'].append(1)
    else:
      data['tag'].append(2)
    words = []
    langs = []
    for wordl in entry[1:]:
      word, lang = wordl.split('\t')
      try:
        if word not in fields and word[0] not in fields:
          words.append(word)
          langs.append(lang)
      except:
        print(word)
        break
    data['text'].append(' '.join(words))
    data['lang'].append(' '.join(langs))
  return data

In [4]:
with open(datapath+'train.txt', 'r') as f:
  trainval = create_data(f)
with open(datapath+'test.txt', 'r') as f:
  test = create_data(f)

In [0]:
test = pd.DataFrame(test)
trainval = pd.DataFrame(trainval)

In [0]:
SPLIT = 0.1
MAXLEN_WORD = 128
MAXLEN_SENT = 64
EMB_FEATURES = 256

In [0]:
train, val = train_test_split(trainval, test_size = SPLIT, random_state = 42)

In [0]:
tokenizer_lang = Tokenizer(num_words=4, filters='')
tokenizer_lang.fit_on_texts(train['lang'])

In [0]:
tokenizer_text = Tokenizer(num_words=EMB_FEATURES+3, filters='', char_level=True)
tokenizer_text.fit_on_texts(train['text'])

In [0]:
def create_3d_data(data, tokenizer_text, tokenizer_lang, MAXLEN_WORD, MAXLEN_SENT):
  vectorizer = TfidfVectorizer(max_features=5000)
  X_text = vectorizer.fit_transform(data['text'])
  X_lang = sequence.pad_sequences(tokenizer_lang.texts_to_sequences(data['lang']), maxlen=MAXLEN_SENT)
  Y = np.array(data['tag'])
  return X_text, X_lang, Y

In [0]:
X_text_train, X_lang_train, Y_train = create_3d_data(train, tokenizer_text, tokenizer_lang, MAXLEN_WORD, MAXLEN_SENT)
X_text_val, X_lang_val, Y_val = create_3d_data(val, tokenizer_text, tokenizer_lang, MAXLEN_WORD, MAXLEN_SENT)

In [0]:
X_text_test, X_lang_test, Y_test = create_3d_data(test, tokenizer_text, tokenizer_lang, MAXLEN_WORD, MAXLEN_SENT)

In [13]:
print(X_text_train.shape, X_lang_train.shape, Y_train.shape)

(13618, 5000) (13618, 64) (13618,)


In [14]:
print(X_text_test.shape, X_lang_test.shape, Y_test.shape)

(1870, 5000) (1870, 64) (1870,)


In [15]:
np.max(X_text_train)

1.0

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

In [18]:
logreg.fit(X_text_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='multinomial', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
def cal_accuracy(y_test, y_pred): 
      
    print("Confusion Matrix: \n", 
        confusion_matrix(y_test, y_pred)) 
      
    print ("Accuracy : ", 
    accuracy_score(y_test,y_pred)*100) 
      
    print("Report \n: ", 
    classification_report(y_test, y_pred)) 

In [0]:
pred = logreg.predict(X_text_val)

In [21]:
cal_accuracy(Y_val, pred)

Confusion Matrix: 
 [[189 120 160]
 [213  92 165]
 [238 135 202]]
Accuracy :  31.90224570673712
Report 
:                precision    recall  f1-score   support

           0       0.30      0.40      0.34       469
           1       0.27      0.20      0.23       470
           2       0.38      0.35      0.37       575

    accuracy                           0.32      1514
   macro avg       0.31      0.32      0.31      1514
weighted avg       0.32      0.32      0.31      1514



In [0]:
pred = logreg.predict(X_text_test)

In [23]:
cal_accuracy(Y_test, pred)

Confusion Matrix: 
 [[152 110 320]
 [155  92 286]
 [230 140 385]]
Accuracy :  33.63636363636363
Report 
:                precision    recall  f1-score   support

           0       0.28      0.26      0.27       582
           1       0.27      0.17      0.21       533
           2       0.39      0.51      0.44       755

    accuracy                           0.34      1870
   macro avg       0.31      0.31      0.31      1870
weighted avg       0.32      0.34      0.32      1870

